**Napomena:** SQL upite i skripte možete sačuvati kao .sql fajlove i izvršavati ih u DBeaver SQL editoru.

### 1. Kreiranje izvora podataka

Napravite source koji će čitati poruke sa Kafka topica `user_events` iz prethodne vježbe.

In [ ]:
CREATE SOURCE IF NOT EXISTS user_event_source(
    timestamp TIMESTAMP,
    user_id INT,
    action VARCHAR,
    bank VARCHAR,
    transaction_type VARCHAR,
    amount REAL,
    message_key VARCHAR
)
WITH(
    connector='kafka',
    topic='user_events',
    properties.bootstrap.server='redpanda-0:9092'
)FORMAT PLAIN ENCODE JSON;

### 2. Kreiranje materijalizovanih pogleda

Kreirajte materijalizovane poglede na osnovu podataka iz `user_events_source`, tako da svaki tip poruka ima odvojen pogled. Jedan za activity tip poruka (`activity_data`) i jedan za transactions tip poruka (`transaction_data`).

In [ ]:
CREATE MATERIALIZED VIEW activity_data AS
SELECT *
FROM user_events_source
WHERE message_type = 'activity_data';

CREATE MATERIALIZED VIEW transaction_data AS
SELECT *
FROM user_events_source
WHERE message_type = 'transaction_data';

### 3. Agregiranje podataka po korisniku

Nakon što ste kreirali poglede, potrebno je da obavite agregaciju podataka iz `activity_data` i `transaction_data`. Kreirajte sljedeće materijalizovane poglede:

3.1. Broj akcija po korisniku (user_activity_count)  
Kreirati materijalizovani pogled koji prikazuje ukupan broj akcija za svakog korisnika, bez obzira na tip akcije.


In [ ]:
CREATE MATERIALIZED VIEW user_activity_count AS
SELECT user_id, COUNT(*) AS total_actions
FROM activity_data
GROUP BY user_id;

3.2. Agregiranje aktivnosti korisnika (user_activity_summary)  
Kreirati materijalizovani pogled koji prikazuje statistiku aktivnosti po korisniku. Za svakog korisnika potrebno je izračunati:  
- Broj uzimanja proizvoda (`take_product`)  
- Broj vraćanja proizvoda (`leave_product`)  
- ... 

In [ ]:
CREATE MATERIALIZED VIEW user_activity_summary AS
SELECT user_id,
       COUNT(CASE WHEN action = 'take_product' THEN 1 END) AS take_product_count,
       COUNT(CASE WHEN action = 'leave_product' THEN 1 END) AS leave_product_count
FROM activity_data
GROUP BY user_id;

In [ ]:
CREATE MATERIALIZED VIEW user_activity_summary AS
SELECT user_id,
       COUNT(*) FILTER (WHERE action = 'take_product') AS take_product_count,
       COUNT(*) FILTER (WHERE action = 'leave_product') AS leave_product_count,
FROM activity_data
GROUP BY user_id;

3.3. Pregled transakcija po korisniku (user_transaction_summary)  
Kreirati materijalizovani pogled koji sumira finansijske aktivnosti korisnika. Za svakog korisnika potrebno je izračunati:  
- Broj i ukupan iznos depozita  
- Broj i ukupan iznos povlačenja sredstava
- ... 

### 4. Spajanje podataka

Kreirajte materijalizovani pogled koji sadrži sve informacije izračunate prethodnim agregiranjem: `user_activity_summary` i `user_transaction_summary`.

### 5. Praćenje posljednje aktivnosti korisnika (latest_activity_per_user)
Kreirati materijalizovani pogled koji za svakog korisnika prikazuje njegov posljednji zapis aktivnosti.   
Za svaki zapis čuvaju se svi detalji aktivnosti, a izbor vrši na osnovu najnovijeg timestamp-a po korisniku.

### 6. TUMBLE funkcija

6.1. Generisanje vremenskih prozora   
Kreirati upit koji dijeli podatke iz `activity_data` u vremenske prozore od 5 sekundi, koristeći TUMBLE funkciju.

6.2. Filtriranje najnovijeg vremenskog prozora  
Prikazati sve aktivnosti koje pripadaju posljednjem vremenskom prozoru.

6.3. Agregacija po vremenskim prozorima  
Izračunati broj aktivnosti po korisniku u svakom vremenskom prozoru.

### 7. Definisanje funkcije za izvršavanje SQL upita

Definisati funkciju `execute_query` koja izvršava SQL upit nad zadatom bazom. Koristiti `psycopg2` biblioteku za povezivanje na bazu i izvršavanje upita.  
Izvršiti prethodne upite koristeći ovu funkciju.

In [ ]:
# code


### Zadatak: Kreiranje RisingWave utils modula

Kreirati Python modul `rw_utils.py` koji će sadržati pomoćne funkcije za rad sa RisingWave-om koristeći psycopg2 paket.

*Proučiti [psycopg - PostgreSQL database adapter for Python dokumentaciju](https://www.psycopg.org/docs/).*

Modul treba da sadrži:
- Funkciju za povezivanje na bazu
- Jednu ili više funkcija za izvršavanje SQL upita